In [1]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, filterFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, IntSlider
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

# login to myplant()
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

In [2]:
############ START BOILERPLATE ################ Interactivity -- BOKEH
import os 
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "13vjb0v8h4e84vlc5690j8b81ssrfpa2cv8omcij8ip4lt1kq242"
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.io import curdoc, show
from bokeh.layouts import column, row, gridplot, layout
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool, Div
from bokeh.models.widgets import Slider, TextInput, Button, CheckboxGroup
from bokeh.io import output_notebook # enables plot interface in J notebook

# init bokeh

from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler


output_notebook()
############ END BOILERPLATE ############

Loading BokehJS ...

In [3]:
# Select Engine
lookup = 'Forsa Hartmoor'
motor_num = 0
def sfun(x):
    return all([ (lookup in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T;
motor = fleet.iloc[motor_num]

modes = ['undefined','OFF','MAN','AUTO']
success = [True,False]
e=Engine.from_fleet(mp,motor)
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

('Forsa Hartmoor M01', ['undefined', 'OFF', 'MAN', 'AUTO'], [True, False])

In [5]:
# select Time Frame & # Run State Machines
#pp_from=e['Commissioning Date']
pp_from='2022-03-01'
#pp_to=datetime.now()
pp_to='2022-04-01'
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm.run0(enforce=True, silent=False, debug=False)
fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm.run2(silent = False, debug = False)

FSM2: 100%|████████████████████████████████| 55/55 [00:03<00:00, 16.13 starts/s]


In [6]:
rdf = rda = fsm.starts
rdb = rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))].reset_index(drop='index')
#print(f"Starts: {rdf.shape[0]}, Successful: {rdf[rdf['success'] == True].shape[0]}, Failed: {rdf[rdf['success'] == False].shape[0]} => {rdf[rdf['success'] == True].shape[0]/rdf.shape[0]*100.0:3.1f}%")

In [7]:
startversuch = rdb.iloc[0]
disp_result(startversuch)
pd.DataFrame.from_dict(e.dash, orient='index').T

no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,True,undefined,102.492999,4.336999,21.489,4.344999,58.834,87.017999,278.515996,4515.612,1.154457,7221.012999,126.793,9.994999,50.68,0,0


,Name,Engine ID,Design Number,Engine Type,Engine Version,P,P_nom,BMEP,serialNumber,id,Count_OpHour,val start,oph@start,oph parts,LOC
0,Forsa Hartmoor M01,M01,AL87,624,H12,24,4495.0,24.5,1486144,159396,281.0,2022-01-10,63,218.0,0.145


In [8]:
# PLotter
def myfigures(e):
    return {
    'actors' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['Ignition_ITPAvg'],'ylim': [-10, 30], 'color':'rgba(255,0,255,0.4)', 'unit':'°KW'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':['Various_Values_PosThrottle','Various_Values_PosTurboBypass'],'ylim': [-10, 110], 'color':['rgba(105,105,105,0.6)','rgba(165,42,42,0.4)'], 'unit':'%'},
    ],
    'lubrication' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
    {'col':['Hyd_PressOilDif'],'ylim': [0, 2], 'color':'black', 'unit': 'bar'},
    {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown', 'unit': 'bar'},
    {'col':['Hyd_TempOil','Hyd_TempCoolWat','Hyd_TempWatRetCoolOut'],'ylim': [0, 110], 'color':['#2171b5','orangered','hotpink'], 'unit':'°C'},
    ],
    'exhaust' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':e.dataItemsCyl('Exhaust_TempCyl*'),'ylim': [400, 700], 'unit':'°C'},
    {'col':e.dataItemsCyl('Knock_Valve_Noise_Cyl*'),'ylim': [0, 4000], 'unit':'V'},
    ],
    'ignition' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':e.dataItemsCyl('Monic_VoltCyl*'),'ylim': [0, 100], 'unit':'kV'},
    {'col':e.dataItemsCyl('Ignition_ITPCyl*'),'ylim': [0, 40], 'unit':'°KW'},
    {'col':e.dataItemsCyl('Knock_KLS98_IntKnock_Cyl*'),'ylim': [-30, 60], 'unit':'%'},
    ],  
}
lfigures = myfigures(fsm._e)
plotdef, vset = cplotdef(mp, lfigures)

In [9]:
#dmaxlength = 1800
dmaxlength = None
dminlength = None
#startversuch = rdb.iloc[vv]
ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset)
data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
source = ColumnDataSource(data)
for doplot in plotdef[:1]:
    dset = lfigures[doplot]
    ltitle = f"{ftitle} | {doplot}"
    try:
        if count_columns(dset) > 12: # no legend, if too many lines.
            fig = FSM_splotBC(fsm, startversuch, source, dset, title=ltitle, legend=False, figsize=(18, 10))
        else:
            fig = FSM_splotBC(fsm, startversuch, source, dset, title=ltitle, figsize=(18, 10))

        fig = FSM_add_Notations(fig, fsm, startversuch)
        disp_alarms(startversuch)
        disp_warnings(startversuch)
        fig = FSM_add_Alarms(fig, fsm, startversuch)
        fig = FSM_add_Warnings(fig, fsm, startversuch)
        #bokeh_show(fig)
    except Exception as err:
        print(err)

von: 01.03.2022 07:55:05 bis: 01.03.2022 10:03:12


In [10]:
# # Set up data
# N = 200
# x = np.linspace(0, 4*np.pi, N)
# y = np.sin(x)
# source = ColumnDataSource(data=dict(x=x, y=y))

# # Set up plot
# plot = figure(plot_height=600, plot_width=800, title="my sine wave",
#               tools="crosshair,pan,reset,save,wheel_zoom",
#               x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

# plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6);
# #show(plot)

In [11]:

# Set up widgets

#LABELS = ['Timings','Warnings','Alarms','Devices']
#checkbox_group = CheckboxGroup(labels=LABELS, active=[0, 1])
#text = TextInput(title="title", value='my sine wave')
no_startversuch = Slider(title='Startversuch', value=0, start=0, end=1277, step=1)
# offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
# amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
# phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
# freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)
but_up = Button(label='^', button_type='primary', width=50)
but_down = Button(label='v', button_type='warning', width=50)

# Set up callbacks
# def update_title(attrname, old, new):
#     fig.title.text = text.value

def update_up(event):
    no_startversuch.value += 1

def update_down(event):
    no_startversuch.value -= 1

def update_data(attrname, old, new):
    # Get the current slider values
    # a = amplitude.value
    # b = offset.value
    # w = phase.value
    # k = freq.value
    sv = no_startversuch.value_throttled
    startversuch = rdb.iloc[sv]
    disp_result(startversuch)

    # source
    # # Generate the new curve
    # x = np.linspace(0, 4*np.pi, N)
    # y = a*np.sin(k*x + w) + b

    # source.data = dict(x=x, y=y)
    # ### I thought I might need a show() here, but it doesn't make a difference if I add one
    # # show(layout)

    #data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset)
    #data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
    #source.data = data

#def update_checkbox(active):
#    plot.title.text = 'checkbox'

no_startversuch.on_change('value', update_data)
but_up.on_click(update_up)
but_down.on_click(update_down)

#checkbox_group.on_change('name', update_checkbox)

In [12]:

# Set up layouts and add to document
#inputs = bokeh.models.Column(text, offset, amplitude, phase, freq, but_up, but_down)
layout = row(fig, bokeh.models.Column(no_startversuch, row(but_up, but_down)))

def modify_doc(doc):
    doc.add_root(row(layout, width=800))
    doc.title = "Sliders"
    #text.on_change('value', update_title)

handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)

no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,True,undefined,102.492999,4.336999,21.489,4.344999,58.834,87.017999,278.515996,4515.612,1.154457,7221.012999,126.793,9.994999,50.68,0,0


no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,True,undefined,102.492999,4.336999,21.489,4.344999,58.834,87.017999,278.515996,4515.612,1.154457,7221.012999,126.793,9.994999,50.68,0,0


no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,True,undefined,102.492999,4.336999,21.489,4.344999,58.834,87.017999,278.515996,4515.612,1.154457,7221.012999,126.793,9.994999,50.68,0,0


no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,True,undefined,102.492999,4.336999,21.489,4.344999,58.834,87.017999,278.515996,4515.612,1.154457,7221.012999,126.793,9.994999,50.68,0,0


no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,True,undefined,102.492999,4.336999,21.489,4.344999,58.834,87.017999,278.515996,4515.612,1.154457,7221.012999,126.793,9.994999,50.68,0,0


...

no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,True,undefined,102.492999,4.336999,21.489,4.344999,58.834,87.017999,278.515996,4515.612,1.154457,7221.012999,126.793,9.994999,50.68,0,0


no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,True,undefined,102.492999,4.336999,21.489,4.344999,58.834,87.017999,278.515996,4515.612,1.154457,7221.012999,126.793,9.994999,50.68,0,0


no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,True,undefined,102.492999,4.336999,21.489,4.344999,58.834,87.017999,278.515996,4515.612,1.154457,7221.012999,126.793,9.994999,50.68,0,0


no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,True,undefined,102.492999,4.336999,21.489,4.344999,58.834,87.017999,278.515996,4515.612,1.154457,7221.012999,126.793,9.994999,50.68,0,0


ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'event_name': 'button_click', 'event_values': {}}}], 'references': []} 
 error: IndexError('single positional indexer is out-of-bounds')
Traceback (most recent call last):
  File "/Users/dieterchvatal/opt/anaconda3/lib/python3.9/site-packages/bokeh/server/protocol_handler.py", line 97, in handle
    work = await handler(message, connection)
  File "/Users/dieterchvatal/opt/anaconda3/lib/python3.9/site-packages/bokeh/server/session.py", line 93, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
  File "/Users/dieterchvatal/opt/anaconda3/lib/python3.9/site-packages/bokeh/server/session.py", line 287, in _handle_patch
    message.apply_to_document(self.document, self)
  File "/Users/dieterchvatal/opt/anaconda3/lib/python3.9/site-packages/bokeh/protocol/messages/patch_doc.py", line 115, in apply_